In [3]:
import json

filepath = r'../production_data_without_summary.json'

with open(filepath, 'r') as f:
    data = json.load(f)


In [2]:
import os
os.getcwd()

'/home/sosa.s/gaitor-function-calling/data/ukey'

In [20]:
data[27]

{'input': [{'chatgptMessage': {'role': 'user',
    'content': 'linkedin hashtags for tech'},
   'functions': [{'name': 'getHashtagFollowerCount',
     'description': 'Get the follower count, follower count over time and related trending hashtags of a specified LinkedIn hashtag',
     'parameters': {'type': 'object',
      'properties': {'path_params': {'type': 'object',
        'properties': {'hashtag': {'type': 'string',
          'description': 'The specified hashtag.'}},
        'required': ['hashtag']}}}}],
   'timeSent': '2023-09-20T20:08:35.342Z'}],
 'target': {'chatgptMessage': {'content': None,
   'function_call': {'arguments': '{}', 'name': 'getHashtagFollowerCount'},
   'role': 'assistant'},
  'functions': [{'name': 'getHashtagFollowerCount',
    'description': 'Get the follower count, follower count over time and related trending hashtags of a specified LinkedIn hashtag',
    'parameters': {'type': 'object',
     'properties': {'path_params': {'type': 'object',
       'prope

In [5]:
ex_data_basic_res = data[12]
ex_data_fc_res = {'input': [{'chatgptMessage': {'role': 'user',
    'content': 'sumarize this in 1 sentence https://openai.com/blog/function-calling-and-other-api-updates'},
   'functions': [{'name': 'transcodeWebPage',
     'description': 'Acquire precise webpage details or real-time search engine responses based on user-input content.',
     'parameters': {'type': 'object',
      'properties': {'json': {'properties': {'link': {'type': 'string',
          'description': 'This parameter takes either a URL or a non-URL string. If a URL is given, the model will engage with the designated webpage to collect or interact with its data. If a non-URL string is given, the model will handle it as a search inquiry and try to find related real-time news or information. To guarantee the best results, make sure the input is a valid URL or a succinct search query.'}},
        'type': 'object'}}}}],
   'timeSent': '2023-09-20T17:23:29.989Z'}],
 'target': {'chatgptMessage': {'content': None,
   'function_call': {'arguments': '{"json": {"link": "https://openai.com/blog/function-calling-and-other-api-updates"}}',
    'name': 'transcodeWebPage'},
   'role': 'assistant'},
  'functions': [{'name': 'transcodeWebPage',
    'description': 'Acquire precise webpage details or real-time search engine responses based on user-input content.',
    'parameters': {'type': 'object',
     'properties': {'json': {'properties': {'link': {'type': 'string',
         'description': 'This parameter takes either a URL or a non-URL string. If a URL is given, the model will engage with the designated webpage to collect or interact with its data. If a non-URL string is given, the model will handle it as a search inquiry and try to find related real-time news or information. To guarantee the best results, make sure the input is a valid URL or a succinct search query.'}},
       'type': 'object'}}}}],
  'timeSent': '2023-09-20T17:23:35.166Z'}}

def to_instruction(data_ins):
  ex_instruction = """
  Below is an instruction that describes a "function calling" task...

  ### Instruction:
  Analyze the prompt and json speicifaction pair (denoted in <FC></FC>) to produce a relevant "function calling" response (denoted in <FCR></FCR>), otherwise return a plain text response.

  ### Input:
  sumarize this in 1 sentence https://openai.com/blog/function-calling-and-other-api-updates
  <FC>""" + json.dumps(data_ins["input"][-1]["functions"]) + """</FC>
  ### Response:
  """
  if ("function_call" in data_ins["target"]["chatgptMessage"]):
    ex_instruction += "<FCR>" + json.dumps(data_ins["target"]["chatgptMessage"]["function_call"]) + "</FCR>"
  else:
    ex_instruction += data_ins["target"]["chatgptMessage"]["content"]
  return ex_instruction

print(to_instruction(ex_data_fc_res))


  Below is an instruction that describes a "function calling" task...

  ### Instruction:
  Analyze the prompt and json speicifaction pair (denoted in <FC></FC>) to produce a relevant "function calling" response (denoted in <FCR></FCR>), otherwise return a plain text response.

  ### Input:
  sumarize this in 1 sentence https://openai.com/blog/function-calling-and-other-api-updates
  <FC>[{"name": "transcodeWebPage", "description": "Acquire precise webpage details or real-time search engine responses based on user-input content.", "parameters": {"type": "object", "properties": {"json": {"properties": {"link": {"type": "string", "description": "This parameter takes either a URL or a non-URL string. If a URL is given, the model will engage with the designated webpage to collect or interact with its data. If a non-URL string is given, the model will handle it as a search inquiry and try to find related real-time news or information. To guarantee the best results, make sure the input is a

In [8]:
train = []
with open('./production_train.json', 'a') as outfile:
    for example in data:
        try:
            text = to_instruction(example)
            train.append({"text": text})
        except:
            pass
    json.dump(train, outfile)

In [23]:
print(train[27]["text"])


  Below is an instruction that describes a "function calling" task...

  ### Instruction:
  Analyze the prompt and json speicifaction pair (denoted in <FC></FC>) to produce a relevant "function calling" response (denoted in <FCR></FCR>), otherwise return a plain text response.

  ### Input:
  sumarize this in 1 sentence https://openai.com/blog/function-calling-and-other-api-updates
  <FC>[{"name": "getHashtagFollowerCount", "description": "Get the follower count, follower count over time and related trending hashtags of a specified LinkedIn hashtag", "parameters": {"type": "object", "properties": {"path_params": {"type": "object", "properties": {"hashtag": {"type": "string", "description": "The specified hashtag."}}, "required": ["hashtag"]}}}}]</FC>
  ### Response:
  <FCR>{"arguments": "{}", "name": "getHashtagFollowerCount"}</FCR>
